In [3]:
import os, glob 
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import choice, sample, seed

In [4]:
seed(42)
img_fps = sample(glob.glob("/home/adelb/Documents/Bpartners/Pleiades/dataset/bati_2014_cherbourg/images/*.png"), 200)

In [12]:
def average_histogram_year(references):
    """Compute the average color distribution from multiple images."""
    imgs = [cv2.cvtColor(cv2.imread(ref), cv2.COLOR_BGR2LAB) for ref in references]
    stds = [img.std(axis=(0,1)) for img in imgs]
    means = [img.mean(axis=(0,1)) for img in imgs]
    stds= np.stack(stds, axis=0)
    means = np.stack(means, axis= 0)
    
    return means.mean(axis=0), stds.mean(axis=0)

def color_transfer(source, LAB_means, LAB_stds, BGR = False ):
    """
    Transfers color distribution from the target to the source.
    """
    # Convert images to LAB color space
    src_lab = cv2.cvtColor(source, cv2.COLOR_RGB2LAB).astype(np.float32)
    if BGR :
        src_lab = cv2.cvtColor(source, cv2.COLOR_BGR2LAB).astype(np.float32)
    l_src, a_src, b_src = cv2.split(src_lab)
    
    
    # Mean and std for each channel of source
    src_mean, a_src_mean, b_src_mean = src_lab.mean(axis=(0,1))
    src_std, a_src_std, b_src_std = src_lab.std(axis=(0,1))
    
    # Mean and std for each channel of target
    tar_mean, a_tar_mean, b_tar_mean = LAB_means
    tar_std, a_tar_std, b_tar_std = LAB_stds

    # Perform color transfer
    l_new = (l_src - src_mean) * (tar_std / src_std) + tar_mean
    a_new = (a_src - a_src_mean) * (a_tar_std / a_src_std) + a_tar_mean
    b_new = (b_src - b_src_mean) * (b_tar_std / b_src_std) + b_tar_mean
    
    # Clip to valid range
    l_new = np.clip(l_new, 0, 255)
    a_new = np.clip(a_new, 0, 255)
    b_new = np.clip(b_new, 0, 255)

    # Combine back into LAB
    transfer_lab = cv2.merge([l_new, a_new, b_new]).astype(np.uint8)

    # Convert back to RGB
    transfer = cv2.cvtColor(transfer_lab, cv2.COLOR_LAB2RGB)
    if BGR:
        transfer = cv2.cvtColor(transfer_lab, cv2.COLOR_LAB2BGR)

    return transfer

In [13]:
lab_means, lab_stds = average_histogram_year(img_fps)
out_profile = "/home/adelb/Documents/Bpartners/Pleiades/dataset/bati_2014_cherbourg/cherbourg_2014_profile_2.npz"
np.savez(out_profile, lab_means=lab_means, lab_stds=lab_stds)

In [18]:
ref_folder = "/home/adelb/Documents/Bpartners/Pleiades/dataset/bati/test/images"
out_folder = "/home/adelb/Documents/Bpartners/Pleiades/dataset/bati/test/images"
os.makedirs(out_folder, exist_ok=True)
for fn in os.listdir(ref_folder):
    img = cv2.imread(f"{ref_folder}/{fn}")
    n_img = color_transfer(img, lab_means, lab_stds, True)
    cv2.imwrite(f"{out_folder}/{fn}", n_img)

117

In [8]:
import os
import cv2
import numpy as np
from skimage import exposure

# ---------- Helpers ----------

def compute_cdf(img_gray):
    """Compute normalized CDF for grayscale image."""
    hist, bins = np.histogram(img_gray.flatten(), 256, [0,256])
    cdf = hist.cumsum()
    cdf = cdf / cdf[-1]  # normalize
    return cdf

def match_cdf(src_gray, target_cdf):
    """Histogram specification: match src_gray to target_cdf."""
    src_hist, bins = np.histogram(src_gray.flatten(), 256, [0,256])
    src_cdf = src_hist.cumsum()
    src_cdf = src_cdf / src_cdf[-1]

    # build LUT by matching closest CDF values
    lut = np.zeros(256, dtype=np.uint8)
    for i in range(256):
        diff = np.abs(src_cdf[i] - target_cdf)
        lut[i] = np.argmin(diff)

    return cv2.LUT(src_gray, lut)

def apply_texture(img, texture=0.3):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB).astype(np.float32)
    
    # Texture enhancement (local contrast)
    L = lab[:,:,0].astype(np.uint8)
    blurred = cv2.GaussianBlur(L, (0,0), 3)
    L_tex = cv2.addWeighted(L, 1 + texture, blurred, -texture, 0)
    lab[:,:,0] = np.clip(L_tex, 0, 255)
    
    out = cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR)
    return out

# ---------- Step 1: Build profile ----------

def build_profile(preprocessed_folder, out_file="profile.npz"):
    means, stds, cdfs, sharps = [], [], [], []

    for fname in os.listdir(preprocessed_folder):
        path = os.path.join(preprocessed_folder, fname)
        img = cv2.imread(path)
        if img is None:
            continue

        # LAB stats
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB).astype(np.float32)
        means.append(lab.mean(axis=(0,1)))
        stds.append(lab.std(axis=(0,1)))

        # luminance CDF (from L channel)
        L = lab[:,:,0].astype(np.uint8)
        cdfs.append(compute_cdf(L))

        # sharpness

    profile = {
        "lab_means": np.mean(means, axis=0),
        "lab_stds": np.mean(stds, axis=0),
        "cdf_mean": np.mean(cdfs, axis=0),
    }
    np.savez(out_file, **profile)
    print(f"Profile saved to {out_file}")

# ---------- Step 2: Apply profile ----------

def apply_profile(img, profile_file="profile.npz", texture=.1):
    prof = np.load(profile_file)

    lab_means = prof["lab_means"]
    lab_stds = prof["lab_stds"]
    cdf_mean = prof["cdf_mean"]
    print(cdf_mean.shape)


    # --- Color transfer (LAB mean/std) ---
    src_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB).astype(np.float32)
    src_mean = src_lab.mean(axis=(0,1))
    src_std = src_lab.std(axis=(0,1))

    l_new = (src_lab[:,:,0] - src_mean[0]) * (lab_stds[0]/src_std[0]) + lab_means[0]
    a_new = (src_lab[:,:,1] - src_mean[1]) * (lab_stds[1]/src_std[1]) + lab_means[1]
    b_new = (src_lab[:,:,2] - src_mean[2]) * (lab_stds[2]/src_std[2]) + lab_means[2]

    l_new = np.clip(l_new, 0, 255).astype(np.uint8)
    a_new = np.clip(a_new, 0, 255).astype(np.uint8)
    b_new = np.clip(b_new, 0, 255).astype(np.uint8)

    lab_new = cv2.merge([l_new, a_new, b_new])
    img_colored = cv2.cvtColor(lab_new, cv2.COLOR_LAB2BGR)

    # --- Exposure match (histogram specification of L channel) ---
    lab2 = cv2.cvtColor(img_colored, cv2.COLOR_BGR2LAB)
    L = lab2[:,:,0]
    L_matched = match_cdf(L, cdf_mean)
    lab2[:,:,0] = L_matched
    img_exposure = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

    # --- Texture match (sharpness) ---
    final_img = apply_texture(img_exposure, texture=texture)

    return final_img if texture > 0 else img_exposure 

Profile saved to /home/adelb/Documents/Bpartners/Pleiades/dataset/bati_2014_cherbourg/cherbourg_2014_profile.npz


In [ ]:
ref_folder = "/home/adelb/Documents/Bpartners/Pleiades/dataset/bati_2014_cherbourg/images"
profile_path = "/home/adelb/Documents/Bpartners/Pleiades/dataset/bati_2014_cherbourg/cherbourg_2014_profile.npz"
build_profile(ref_folder, profile_path)

In [11]:
out_folder = "/home/adelb/Documents/Bpartners/Pleiades/dataset/bati_2014_cherbourg/n_images"
os.makedirs(out_folder, exist_ok=True)

for fn in os.listdir(ref_folder):
    img = cv2.imread(f"{ref_folder}/{fn}")
    n_img = apply_profile(img, profile_path, texture=0)
    cv2.imwrite(f"{out_folder}/{fn}", n_img)

(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)
(256,)